<a href="https://colab.research.google.com/github/sohamhmhm/DevD/blob/main/DEVD_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets transformers torch pillow

In [6]:
import torch
import transformers
from transformers import CLIPProcessor, CLIPModel, T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import os

In [7]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

GPU Available: True
GPU Name: Tesla T4


In [8]:
dataset = load_dataset("HuggingFaceM4/WebSight", split="train", streaming=True)
small_dataset = dataset.take(500)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

In [9]:
# ✅ Define device before using it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [10]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [18]:
def preprocess(sample):
    image = sample["image"].convert("RGB").resize((256, 256), Image.LANCZOS)  # Convert to RGB & resize

    # Convert the image to a PyTorch tensor
    inputs = clip_processor(images=image, return_tensors="pt")
    pixel_values = inputs["pixel_values"].squeeze(0)

    # Handle missing `html` key
    html = sample.get("html", "<html><body>No HTML found</body></html>")
    html_tokens = tokenizer(html, return_tensors="pt", padding="max_length", truncation=True)
    input_ids = html_tokens["input_ids"].squeeze(0)  # Extract tensor

    # ✅ Return only the necessary tensors, not the original image
    return {"pixel_values": pixel_values, "input_ids": input_ids}

In [1]:
train_data = list(small_dataset.map(preprocess))

In [16]:
def train_model(model, train_data, epochs=3):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()

    train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True, num_workers=0)

    for epoch in range(epochs):
        for batch in train_dataloader:
            images = batch["pixel_values"].to(device)
            labels = batch["input_ids"].to(device)

            outputs = model(pixel_values=images, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    return model

In [17]:
trained_model = train_model(t5_model, train_data)

In [ ]:
trained_model.save_pretrained("image-to-html-model")
tokenizer.save_pretrained("image-to-html-model")
print("Model saved!")

In [ ]:
def generate_html(image_path):
    image = Image.open(image_path).resize((256, 256), Image.LANCZOS)
    processed_image = clip_processor(images=image, return_tensors="pt")["pixel_values"].to(device)

    generated_ids = trained_model.generate(processed_image)
    html_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return html_output

In [ ]:
from google.colab import files
print("Please upload an image file for HTML generation.")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nGenerating HTML for {filename} ...")
    html_code = generate_html(filename)
    print("Generated HTML Code:\n", html_code)